In [1]:
import yaml

In [2]:
from src.config import DATA_DIR, PARAMS_DIR, RESULTS, ROOT_DIR
import os
print(DATA_DIR)
os.chdir(ROOT_DIR)

Project Directory: /data2/isshamie/time_neutrophils
/data2/isshamie/time_neutrophils/data


In [4]:
config

'parameters/kawasaki/test_run/kawasaki_snakemake_all_nsample1000.yaml'

In [5]:
ls parameters/

kawasaki/                       params_sample_4.yaml*
parameters_snakemake_001.yaml*  params_sample_5.yaml*
params_sample_10.yaml*          params_sample_6.yaml*
params_sample_11.yaml*          params_sample_7.yaml*
params_sample_1.yaml*           params_sample_8.yaml*
params_sample_2.yaml*           params_sample_9.yaml*
params_sample_3.yaml*           plot_cluster_params_sample_2.yaml*


In [7]:
import os
import pandas as pd
from snakemake.utils import validate
import numpy as np
from src import run_umap

configfile = "parameters/kawasaki/test_run/kawasaki_snakemake_all_nsample1000.yaml"
with open(configfile) as file:
    config = yaml.full_load(file)
#from src.utils import logs

# samples = pd.read_table(config["samples"], dtype=str).set_index(["sample"], drop=False)
# RAW_SAMPLES = samples.apply(lambda x: os.path.join(x["raw"], x["bam"]), axis=1)
#print('index',samples.index)


normalizations = config["normalizations"]
reduct_technique = config["reduction"]
n_iters = np.arange(config["n_iters"])
n_subsample = config["n_subsample"]
min_neighbors= config["min_neighbors"]
min_neighbors = list(map(lambda x: int(x), min_neighbors))
min_distances = config["min_distances"]
min_distances = list(map(lambda x: float(x), min_distances))

features = config["features"]
print(features)
cluster = config["cluster_technique"]
min_cluster_size = config["min_cluster_size"]
min_sample = config["min_sample"]

#report: "report/workflow.rst"

prefix = [config["prefix"]]



['Cell Tracker Intensity', 'PI Intensity', 'AnexinV Intensity', 'Cell Aspect Ratio', 'Cell Size', 'Cell Circularity']


In [6]:
config

{'raw_dir': 'raw/kawasaki/FCS Files/',
 'prefix': 'kawasaki/test_run/all_nsample1000',
 'tidy_name': 'tidy',
 'is_kawa': True,
 'wells': None,
 'name': {'start': 'Test Run - ', 'end': '.csv'},
 'features': ['Cell Tracker Intensity',
  'PI Intensity',
  'AnexinV Intensity',
  'Cell Aspect Ratio',
  'Cell Size',
  'Cell Circularity'],
 'normalizations': ['log10_z_out3'],
 'reduction': 'umap',
 'n_subsample': 0.25,
 'n_iters': 3,
 'min_neighbors': [15, 200, 100],
 'min_distances': [0],
 'cluster_technique': ['hdb'],
 'min_cluster_size': [30, 60],
 'min_sample': [100]}

In [ ]:
config

In [ ]:
    input:
        data_f = "data/processed/{prefix}/tidy/fc.tsv",
        meta_f = "data/processed/{prefix}/tidy/meta.tsv"
    output: "data/processed/{prefix}/transform/data_df_{norm}.p"
    params:
        outdir = lambda wildcards, output: os.path.dirname(str(output)),
    shell: "python src/preprocess.py {input.data_f} {params.outdir} {wildcards.norm}"


In [ ]:
    input:
         data_f = "data/processed/{prefix}/transform/data_df_{norm}.p",
         meta_f = "data/processed/{prefix}/tidy/meta.tsv"
    output:
        "results/{prefix}/{norm}/{dim}_out/{min_neighbor}_{min_distance}/embedding_{sim}.p"
    params:
        outdir = lambda wildcards:  f"results/{wildcards.prefix}/{wildcards.norm}/{wildcards.dim}_out", # output: os.path.dirname(str(output)),
        n_iter = config["n_iters"],
        n_subsample = n_subsample,
        #features = "intensity"
    resources:
        mem_mb=50000
    shell: "python src/run_umap.py {input.data_f} {input.meta_f} {params.outdir} {wildcards.min_neighbor} {wildcards.min_distance} {params.n_subsample} --n_iter {params.n_iter} --embed {wildcards.dim}"



In [ ]:
    input:
        data_f = "data/processed/{prefix}/transform/data_df_{norm}.p",
        meta_f = "data/processed/{prefix}/tidy/meta.tsv",
        umap_f = "results/{prefix}/{norm}/{dim}_out/{min_neighbor}_{min_distance}/embedding_{sim}.p"
    output:
        stim_fig = "figures/{prefix}/{norm}/{dim}_out/{min_neighbor}_{min_distance}/sim{sim}/umap_stimuli.png",
    params:
        fig_dir = "figures/{prefix}/{norm}/{dim}_out/{min_neighbor}_{min_distance}/sim{sim}"
    shell: "python src/plot_umap_embeddings.py {input.data_f} {input.meta_f} {input.umap_f} {params.fig_dir} {wildcards.min_neighbor} {wildcards.min_distance}"


In [ ]:
    input:
        umap_f = "results/{prefix}/{norm}/{dim}_out/{min_neighbor}_{min_distance}/embedding_0.p"
    output:
        cluster_f = "results/{prefix}/{norm}/{dim}_out/{min_neighbor}_{min_distance}/{cluster}_out/{min_sample}_{min_cluster_size}.p",
        f_save_fig = "figures/{prefix}/{norm,^[^/]+$}/{dim}_out/{min_neighbor}_{min_distance}/{cluster}_out/{min_sample}_{min_cluster_size}.png"
    resources:
        mem_mb=200000
    shell: "python src/clustering.py {input.umap_f} {output.cluster_f} {output.f_save_fig} {wildcards.cluster} {wildcards.min_sample} {wildcards.min_cluster_size}"
